Study on smaller language : defog/sqlcoder-7b-2 : 7billions parameters

# Requirements

In [1]:
!pip install transformers bitsandbytes accelerate datasets outlines scikit-learn sqlvalidator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.3/343.3 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Load the Model and the dataset

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("defog/sqlcoder-7b-2")
model = AutoModelForCausalLM.from_pretrained("defog/sqlcoder-7b-2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Import the dataset

In [41]:
from datasets import load_dataset
import pandas as pd
#Dataset: https://huggingface.co/datasets/xlangai/spider?row=17
ds = load_dataset("xlangai/spider")
db_schema = pd.DataFrame(load_dataset("richardr1126/spider-schema", split="train"))


<class 'datasets.dataset_dict.DatasetDict'>


In [46]:
print(db_schema.iloc[0])
train = ds["train"]
print(f"Train len {len(train)}")
train = train.select(range(100))  # Ou train = train[:100]
print(type(train))

db_id                                                           perpetrator
Schema (values (type))    perpetrator : Perpetrator_ID (number) , People...
Primary Keys              perpetrator : Perpetrator_ID | people : People_ID
Foreign Keys              perpetrator : People_ID equals people : People_ID
Name: 0, dtype: object
Train len 7000
<class 'datasets.arrow_dataset.Dataset'>


In [43]:
#Only looking for the validation test of the dataset
examples = [
    {"question": ex["question"], "SQL_query": ex["query"], "db_id": ex["db_id"]}
    for ex in ds['validation'].to_list()
]
print(len(examples))
# Split examples and keep only a few samples to have short computation time.
test = examples[:20]
print(f"Test len {len(test)}")
#print(f"Validation len {len(validation_dataset)}")
print(f"First example of test:\n{test[0]}")

1034
Test len 20
First example of test:
{'question': 'How many singers do we have?', 'SQL_query': 'SELECT count(*) FROM singer', 'db_id': 'concert_singer'}
Train len 7000


# Entrainement

In [48]:
def format_example(example):
    # On concatène éventuellement un séparateur entre la question et la requête
    # Ici, on utilise le token de fin du tokenizer pour marquer la séparation
    return {
        "prompt": example["question"] + "\nSQL:",
        "completion": " " + example["query"] + tokenizer.eos_token
    }

train = train.map(format_example)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [51]:
print(train['question'])
print(train['completion'])

['How many heads of the departments are older than 56 ?', 'List the name, born state and age of the heads of departments ordered by age.', 'List the creation year, name and budget of each department.', 'What are the maximum and minimum budget of the departments?', 'What is the average number of employees of the departments whose rank is between 10 and 15?', 'What are the names of the heads who are born outside the California state?', "What are the distinct creation years of the departments managed by a secretary born in state 'Alabama'?", 'What are the names of the states where at least 3 heads were born?', 'In which year were most departments established?', "Show the name and number of employees for the departments managed by heads whose temporary acting value is 'Yes'?", 'How many acting statuses are there?', 'How many departments are led by heads who are not mentioned?', 'What are the distinct ages of the heads who are acting?', "List the states where both the secretary of 'Treasury

In [61]:
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    # Concaténer prompt et completion pour chaque exemple
    full_texts = [p + c for p, c in zip(examples["prompt"], examples["completion"])]
    return tokenizer(full_texts, truncation=True, padding="max_length", max_length=512)


tokenized_train = train.map(tokenize_function, batched=True)

# Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./sqlcoder-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="no",
    logging_dir="./logs",
    learning_rate=2e-5,
    fp16=True,
    push_to_hub=False
)
torch.cuda.empty_cache()
# Créer l'entraîneur
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    tokenizer=tokenizer,
)

# Lancer l'entraînement
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-61-9a0a6f9e8536>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 84.88 MiB is free. Process 51373 has 39.46 GiB memory in use. Of the allocated memory 38.84 GiB is allocated by PyTorch, and 136.18 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [37]:
from peft import get_peft_model, LoraConfig, TaskType
from transformers import TrainingArguments, Trainer

# Configurer LoRA
peft_config = LoraConfig(
    r=16,  # Taille des matrices LoRA
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Appliquer LoRA au modèle
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./sqlcoder-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_dir="./logs",
    learning_rate=2e-5,
    fp16=True,
    push_to_hub=False
)

# Définir l'entraîneur
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    tokenizer=tokenizer,
)

# Lancer l'entraînement
trainer.train()


trainable params: 8,388,608 || all params: 6,746,935,296 || trainable%: 0.1243


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-37-7f5a2e52a86b>:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: You have set `args.eval_strategy` to IntervalStrategy.STEPS but you didn't pass an `eval_dataset` to `Trainer`. Either set `args.eval_strategy` to `no` or pass an `eval_dataset`. 

# Evaluation

In [6]:
#Set the model for evaluation
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32016, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_e

Definition : Generation_config - Prompt  - def generate

In [7]:
from transformers import GenerationConfig

generation_config = GenerationConfig(
  max_new_tokens = 1000,
  min_new_tokens = 50,
  do_sample=False,
  temperature=0.3,
  repetition_penalty=1.0,
  #eos_token_id=None,
  eos_token_id=tokenizer.eos_token_id,
  pad_token_id=tokenizer.pad_token_id,
)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [8]:
def generate(prompt, llm=model, generation_config=generation_config):

  # Create turns with the given prompt
  # Tokenize turns, tokenize the text
  input_tokens = tokenizer(prompt, return_tensors='pt').to('cuda')
  input_ids=input_tokens['input_ids']

  # Ensure we don't use gradient to save memory space and computation time.
  #Generate output tokens
  with torch.no_grad():
    llm = llm.to('cuda')
    outputs = llm.generate(input_ids,
    generation_config=generation_config).to('cuda')

  #decode output token into text

  # Recover and decode answer.
  answer_tokens = outputs[0, input_ids.shape[1]:-1]

  #decode output token into text
  #.strip() delete space useless
  answer = tokenizer.decode(answer_tokens).strip()  # Remove unnecessary spaces

  # Remove the "ASSISTANT: " prefix, if present
  #if answer.startswith("ASSISTANT:"):
    #answer = answer[len("ASSISTANT:"):].strip()  # Remove the prefix and strip spaces

  return answer

Zero shot classification

In [9]:
base_prompt = """
You are an expert in SQL and database querying.
Always return **only** the SQL query, without any explanation, preamble, or additional text.
Your task is to convert natural language question after <<<>>> into optimized SQL queries. Do not provide any other questions.
<<<
Sentence: {sentence} Answer:
>>>
""".strip()

In [10]:
base_prompt_schema = """
You are an expert in SQL and database querying.
Always return **only** the SQL query, without any explanation, preamble, or additional text.
I give you the database schema wich appears in this form : name_of_the_table : Column_name (Type_of_data), ...| table_2 : ....

here's the database schema : {schema}
Your task is to convert natural language question after <<<>>> into optimized SQL queries. Do not provide any other questions.
<<<
Sentence: {sentence} Answer:
>>>
"""

In [11]:
import re

def extract_sql_answer(answer: str):
    # Recherche une ligne qui commence par "SELECT", "INSERT", "UPDATE", ou d'autres mots-clés SQL
    text=answer.replace('\n', ' ').strip()
    match = re.search(r"(SELECT|INSERT|UPDATE|DELETE|CREATE|DROP|ALTER).*", text.strip(), re.IGNORECASE)
    if match:
        query = match.group(0).strip()  # Retourne la première requête SQL trouvée
        query = re.sub(r'<<<.*$', '', query).strip()
        query = re.sub(r';.*$', '', query).strip()
        query = re.sub(r'[;#].*$', '', query).strip()
        return query
    else:
        return text

In [18]:
sql_databases = generate_synthetic_db(db_schema)
print(sql_databases)

>>> ERROR - i=95 / db : world_1 / table : sqlite_sequence
object name reserved for internal use: sqlite_sequence 

CREATE TABLE "sqlite_sequence" (
    "name" TEXT,
    "seq" TEXT
)
INSERT INTO "sqlite_sequence" ("name", "seq") VALUES (?, ?) 

>>> ERROR - i=111 / db : soccer_1 / table : sqlite_sequence
object name reserved for internal use: sqlite_sequence 

CREATE TABLE "sqlite_sequence" (
    "name" TEXT,
    "seq" TEXT
)
INSERT INTO "sqlite_sequence" ("name", "seq") VALUES (?, ?) 

>>> ERROR - i=128 / db : store_1 / table : sqlite_sequence
object name reserved for internal use: sqlite_sequence 

CREATE TABLE "sqlite_sequence" (
    "name" TEXT,
    "seq" TEXT
)
INSERT INTO "sqlite_sequence" ("name", "seq") VALUES (?, ?) 

{'perpetrator': <sqlite3.Cursor object at 0x799a4810eb40>, 'college_2': <sqlite3.Cursor object at 0x799a48172a40>, 'flight_company': <sqlite3.Cursor object at 0x799a48171bc0>, 'icfp_1': <sqlite3.Cursor object at 0x799a48172ac0>, 'body_builder': <sqlite3.Cursor obje

In [31]:
from tqdm import tqdm

predictions=[]
ground_truths=[]
answers=[]
execute_metric=0

for ex in tqdm(test):
  index = db_schema.index[db_schema['db_id'] == ex['db_id']][0]
  prompt = base_prompt_schema.format(sentence=ex["question"], schema = db_schema.loc[index, 'Schema (values (type))'])
  answer = generate(prompt)
  answers.append(answer)

  pred=extract_sql_answer(answer)
  print(pred)
  print(ex["SQL_query"])
  dic=compute_metrics(pred,ex["SQL_query"] , ex['db_id'], sql_databases)
  print(dic['result_metric'])
  execute_metric+=dic['result_metric']
  predictions.append(pred)
  ground_truths.append(ex["SQL_query"])

print(execute_metric/len(test))


  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  5%|▌         | 1/20 [00:02<00:54,  2.89s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


SELECT COUNT(s.Singer_ID) FROM singer s
SELECT count(*) FROM singer
1


 10%|█         | 2/20 [00:05<00:45,  2.53s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


SELECT COUNT(DISTINCT s.Singer_ID) AS total_singers FROM singer s
SELECT count(*) FROM singer
1


 15%|█▌        | 3/20 [00:08<00:45,  2.68s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

SELECT AVG(s.Capacity), MAX(s.Capacity) FROM stadium s
select avg(capacity) ,  max(capacity) from stadium
1


 95%|█████████▌| 19/20 [00:56<00:03,  3.13s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


SELECT s.name, s.capacity FROM stadium s ORDER BY s.average DESC NULLS LAST LIMIT 1
SELECT name ,  capacity FROM stadium ORDER BY average DESC LIMIT 1
1


100%|██████████| 20/20 [00:59<00:00,  2.97s/it]

SELECT s.name, s.capacity FROM stadium s ORDER BY s.average DESC NULLS LAST LIMIT 1
SELECT name ,  capacity FROM stadium ORDER BY average DESC LIMIT 1
1
0.5


In [28]:
print(execute_metric)
print(execute_metric/len(test))

10
0.5


In [ ]:
print(predictions)
print(ground_truths)
compute_metrics(predictions[0], ground_truths[0], ex['db_id'],, sql_databases)

# Génération d'un dataset


In [14]:
import sqlparse
import sqlvalidator
import re
from sqlparse.sql import Identifier, IdentifierList
from sqlparse.tokens import Keyword, DML, Whitespace
from itertools import permutations
import sqlite3
from sqlite3 import Cursor
import random
import string
import pandas as pd

In [15]:
convert_type = {
    "text": "TEXT",
    "number": "INTEGER",
    "time": "TEXT",
    "boolean": "TEXT",
    "others": "TEXT",
}

def generate_synth_data(type_: str, pimary: str):

    nb_elem = 50
    word_lenght = 6
    max_integer = 60

    if type_=="TEXT" and pimary=="PRIMARY KEY":
        unique_strings = set()
        while len(unique_strings) < nb_elem:
            unique_strings.add(''.join(random.choices(string.ascii_uppercase + string.digits, k=word_lenght)))
        return list(unique_strings)

    elif type_=="TEXT" and not pimary=="PRIMARY KEY":
        return [
            ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(word_lenght))
            for _ in range(nb_elem)
        ]

    elif type_=="INTEGER" and not pimary=="PRIMARY KEY":
        return random.choices(list(range(max_integer)), k=nb_elem)

    elif type_=="INTEGER" and  pimary=="PRIMARY KEY":
        return random.sample(list(range(max_integer)), k=nb_elem)

    else:
        print("ERROR TYPE")

def generate_synthetic_db(db_schema: pd.DataFrame) -> dict[str, Cursor]:
    sql_databases: dict[str, Cursor] = {}

    for i, db in db_schema.iterrows():

        conn = sqlite3.connect(":memory:")
        sql_databases[db["db_id"]] = conn.cursor()

        schema = re.split(r'\|', db["Schema (values (type))"])

        p_keys_dict = {}
        for p_key in re.split(r'\|', db["Primary Keys"]):
            if len(p_key) > 0:
                p_key_split = re.split(r':', p_key)
                p_keys_dict[p_key_split[0].strip()] = p_key_split[1].strip()

        for table in schema:
            table_split = re.split(r':', table)
            table_name = table_split[0].strip()

            att_names = []
            att_types = []
            att_synth_data = []

            rows_att = []

            for attribute_type in re.split(r',', table_split[1]):
                if match_ := re.match(r'(.*)\s\((.*)\)', attribute_type):
                    att_name = match_.group(1).strip().replace(" ", "_")
                    att_type = match_.group(2).strip()

                    # Primary Key
                    if table_name in p_keys_dict.keys() and p_keys_dict[table_name] == att_name:
                        primary = "PRIMARY KEY"
                    else:
                        primary = None

                    att_names.append(att_name)
                    att_types.append(att_type)
                    att_synth_data.append(generate_synth_data(convert_type[att_type], primary))

                    rows_att.append(f'"{att_name}" {convert_type[att_type]}{" "+primary if primary is not None else ""}')

            rows_joined = ',\n    '.join(rows_att) # Changement
            create_request = f"CREATE TABLE \"{table_name}\" (\n    {rows_joined}\n)"

            att_names_joined = '", "'.join(att_names) #Changement
            insert_request = f"INSERT INTO \"{table_name}\" (\"{att_names_joined}\") VALUES ({', '.join(['?'] * len(att_names))})"

            try:
                sql_databases[db["db_id"]].execute(create_request)
                sql_databases[db["db_id"]].executemany(insert_request, list(zip(*att_synth_data)))
            except Exception as e:
                print(f'>>> ERROR - i={i} / db : {db["db_id"]} / table : {table_name}') # Changement
                print(e, "\n")
                print(create_request)
                print(insert_request, "\n")

    return sql_databases


# Metrics

In [16]:
def compute_metrics(pred: str, ground_truth: str, db_id: str, sql_databases: dict[str, Cursor]):

    """Compare 2 queries on their semantic

    Return :

    - valid_pred : True if te pred query is semanticaly correct, False otherwise. It is not really reliable.
    - keyword_score [0, 1]: Equivalent of F1 score for SQL keywords presence. The pred query must contains the keywords of the ground truth without adding new keywords.
    - identifier_score [0, 1]: For each keywords, f1 score is computed for identifiers words (table name, attributes...). The average gives the identifier_score.

    """

    def compute_f1score(a: set, b: set, eps: float=1e-4):
        """Equivalent of F1 score metric"""
        TP = len(a & b)
        FP = len(a - b)
        FN = len(b - a)

        return (TP + eps) / (TP + 0.5*(FP + FN) + eps)

    valid_pred = True

    SQL_keywords = ["SELECT", "WHERE", "FROM", "HAVING", "GROUP BY", "LIMIT", "DESC", "ASC"]

    keywords_pred = [word for word in SQL_keywords if word.upper() in pred.upper()]
    keywords_gt = [word for word in SQL_keywords if word.upper() in ground_truth.upper()]

    keyword_score = compute_f1score(set(keywords_pred), set(keywords_gt))

    ### IDENTIFIERS

    dict_identifiers_pred = {kw: set() for kw in SQL_keywords}
    dict_identifiers_gt = {kw: set() for kw in SQL_keywords}

    pattern = r'(' + '|'.join(map(re.escape, SQL_keywords)) + r')'

    # Split text while keeping the keywords in the result
    pred_parts = re.split(pattern, pred.upper())
    gt_parts = re.split(pattern, ground_truth.upper())

    for i, part in enumerate(pred_parts):
        if part in SQL_keywords:
            kw = part
            identifiers = re.split(r'\s+', pred_parts[i+1])
            identifiers = [elem.replace(",", "") for elem in identifiers if elem != ""]
            dict_identifiers_pred[kw] = set(identifiers)

    for i, part in enumerate(gt_parts):
        if part in SQL_keywords:
            kw = part
            identifiers = re.split(r'\s+', gt_parts[i+1])
            identifiers = [elem.replace(",", "") for elem in identifiers if elem.replace(",", "") != ""]
            dict_identifiers_gt[kw] = set(identifiers)

    identifiers_scores = []
    for kw in SQL_keywords:
        if len(dict_identifiers_gt[kw]) + len(dict_identifiers_pred[kw]) > 1:
            identifiers_scores.append(
                compute_f1score(dict_identifiers_pred[kw], dict_identifiers_gt[kw])
            )

    identifiers_score = sum(identifiers_scores)/len(identifiers_scores)

    #### result_metric
    cursor = sql_databases[db_id]

    try:
        cursor.execute(pred)
        results1 = cursor.fetchall()

        cursor.execute(ground_truth)
        results2 = cursor.fetchall()

        result_metric = int(results1==results2)
    except Exception as e:
        print(f"ERROR (db :{db_id}) : {e}")
        result_metric = 0

    return {
        "valid_pred": valid_pred,
        "keyword_score": keyword_score,
        "identifiers_score": identifiers_score,
        "result_metric": result_metric
    }